# PotentioDynamic Scattering Microscopy (PDSM)

## Contrast oscillation analysis
 
Author: ``Sanli Faez (s.faez@uu.nl)``

last update: 27/2/2018
1st version: 14/2/2018

The main use of this notebook is to **test on a small scale** the functions necessary for extracting the scattering contrast oscillations.  

The measurements are done using the a TIR scattering microscope at the surface of a transparent slab while an alternating potential is applied. The setup in Oxford uses labview and creates tdms datafiles. The setup in Utrecht generates HDF5 files. The rest of the analysis should be the same.

### General advice
Use the notebook for testing small datasets, or fractions of larger ones. Once the reliability is verfied, add the code pieces to an script and process batches of data.

#### Addendum
For now, it is har to converge on a very general treatments that can be applied to batches. Data should be analyzed with extensive supervision to avoid misleading results.


### Roadmap
Each step is separately tested and verified.

* Part 1) preprocessing and previewing data
* Part 2) Consructing the contrast oscillation map
* Part 3) Summarizing the sequence in a representative plot
* Part 4) Comparing responses for different conditions


In [7]:
%pylab inline

import sys
sys.path.append('/Users/sanli/Repositories/PDSM') #uncomment including extra files are needed

import matplotlib.pyplot as plt
import numpy as np
import os

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# choose filename and output directory
fdir = r'/Users/sanli/Repositories/Data/PDSM_UnderInvestigation/Shawn/190313/10mM NaCl on ITO_1ms_12mW_1.8Vpp triangle EF 1Hz_200fps/'
mfile = 'event0.tdms'
fext = '.tdms'
fpath = fdir + mfile
print(fpath)
#outputdir = fdir + 'analyzed/' + mfile.strip(fext) + '/'
outputdir = fdir

### Import data from recorded image sequence files
- First next two cells for are HDF5 snap and video (UUTrack). 
For HDF5, you should also choose which sequence from the file you want. Here it's called tracknumber and starts from 0. 

- For TDMS files from Oxford, use the other notebook with _tdms. 


In [ ]:
## run Only for snaps
from PDSM_func import functionsDFSM as dfsm  #functions written and maintained by Kevin 
data = dfsm.ImportHDF5data(fpath)
data.setkey(0)
print("Wherein there is :", data.getkeys())
dset = data[0]['image'] # to read a single snap image
print(dset.shape)
fig = plt.figure(figsize=(16,10))
plt.imshow(np.transpose(np.log10(dset)), vmin = 0.1, vmax=3)
plt.show()
previewfile = outputdir + 'single_snap'
fig.savefig(previewfile)

In [ ]:
# Import HDF5 video sequence:
from PDSM_func import functionsDFSM as dfsm  #functions written and maintained by Kevin 
data = dfsm.ImportHDF5data(fpath)
data.setkey(0)
print("Wherein there is :", data.getkeys())
data.resetkeys()
tracknumber = 4
seq = data[tracknumber,1] #Sanli: what is the second index?
firstline = np.mean(seq[:,20,:], axis=0)
nf = np.size(firstline[firstline >0])
frames = seq[:,:,:nf]
Lx, Ly, nf = frames.shape
print(f"{nf} frames of size {Lx, Ly} in the sequence")

## REMEMBER
do not run this cell if you are importing HDF5 files

In [13]:
print(frames.shape)

(3800, 512, 512)


In [12]:
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
fig = plt.figure(figsize=(16,6))
firstline = np.mean(frames[:,20,:], axis=2)
plt.plot(firstline[:nf])
plt.title("average of a sample line")
plt.xlabel("frame number")
plt.show()
previewfile = outputdir + 'singleline'
# fig.savefig(previewfile)

IndexError: tuple index out of range

### Render some frames and check the drift
Rule of thumb: if the measurement is on stationary sample the average absolute drift is less than 0.01 when the setup is stable.

In [ ]:
nmean = 40
fig = plt.figure(figsize=(12,8))
ax1 = plt.subplot(211)
im = ax1.imshow(np.transpose(np.log10(np.mean(frames[:,:,:nmean],axis=2))), origin ='lower')
plt.colorbar(im, ax = ax1)
plt.title("average intensity")

ax2 = plt.subplot(212)
drift = (frames[:,:,nf-1]-frames[:,:,0])/frames[:,:,0]
im = ax2.imshow(np.transpose(drift), cmap='plasma', vmin=-0.5, vmax=1.0, origin ='lower')
plt.colorbar(im, ax = ax2)
plt.title("r'$\sigma(I)/\sqrt{I}$'")

avg_drift = np.mean(np.abs(drift))
print(f"average relative change is {avg_drift}")
plt.show()
previewfile = outputdir + mfile.strip(fext)+'_drift'
fig.savefig(previewfile)

### IF it is helpful 
Save a couple of images for further reference to the dataset

In [ ]:
# previewfile = outputdir + mfile.strip(fext)+'_rawImage.npy'
# np.save(previewfile, frames[:,:,0])


### Divide the variance by the average to find oscilating spots/patterns
For shot-noise limited background, this valus should be equal to one. Any excess amount means either technical noise (shaking/drift) or induced perturbasion (potentiodynamic contrast) 

In [ ]:
nmean = 600
mean_img = np.mean(frames[:,:,:nmean], axis=2)
dark = 92
mean_img = mean_img - dark + 1

## few lines to look at the differential variation signal rather than the images
# dframes = np.copy(frames)    
# for i in range(nf):
#     dframes[:,:,i] = frames[:,:,i] - mean_img
    
print("average counts:", np.mean(mean_img))

var_img = np.std(frames[:,:,:600], axis=2)
contrast_img = var_img/np.sqrt(mean_img)

# nbins = 40
# hist_img, bin_edges_img = np.histogram(contrast_img, bins = nbins)
# hist_var, bin_edges_var = np.histogram(var_img, bins = nbins)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = plt.subplot(211)
im1 = ax1.imshow(np.transpose(np.log10(mean_img[100:400,:])), origin ='lower')
plt.colorbar(im1, ax = ax1)
plt.title("logarithm of average intensity")

ax2 = plt.subplot(212)
im2 = ax2.imshow(np.transpose(contrast_img[100:400,:]), cmap='plasma', vmax = 8, vmin = 0.9, origin ='lower')
plt.colorbar(im2, ax = ax2)
plt.title(r'$\sigma(I)/\sqrt{I}$')

# ax3 = plt.subplot(223)
# im3 = ax3.semilogy(bin_edges_img[:nbins], hist_img, '.-')
# plt.title("Distribution of |deviation| from mean ")

# ax4 = plt.subplot(224)
# im4 = ax4.semilogy(bin_edges_var[:nbins], hist_var, '.-')
# plt.title("Distribution of standard deviation")

previewfile = outputdir + mfile.strip(fext)+'_contrast.png'
fig.savefig(previewfile)
plt.show()

### Choose one speckle spot to find the oscillation frequency and waveform
To observe a clear oscillation, use the spot with highest variance/mean value. Adding a few neighboring pixels to that spot increases the signal to noise.

In [ ]:
spotmax = np.unravel_index(np.argmax(var_img, axis=None), var_img.shape)
spotmin = np.unravel_index(np.argmin(var_img, axis=None), var_img.shape)

# spotmax = [spotmax[0], spotmax[1]]
print("max variations at ", spotmax)
print("least variations at ", spotmin)
spotmax = (210, 150)
psw = 20 # esitmate fwhm of the speckle peak
specklemax = frames[spotmax[0]-psw:spotmax[0]+psw+1, spotmax[1]-psw:spotmax[1]+psw+1, :] - dark
responsemax = np.sum(np.sum(specklemax, axis = 0), axis = 0)
specklemin = frames[spotmin[0]-psw:spotmin[0]+psw+1, spotmin[1]-psw:spotmin[1]+psw+1, :] - dark
responsemin = np.sum(np.sum(specklemin, axis = 0), axis = 0)

varspotmax = np.std(responsemax)/np.mean(responsemax)
print("relative intensity variation", varspotmax)

fig = plt.figure(figsize=(16,4))
plt.plot(responsemax[:], '.-')
#plt.plot(responsemin)
plt.show()

### Find the Fourier component(s) corresponding to the oscillating potential
This part can be tricky if the oscillations are not prominent. Choose dcbase so that the value at the lower components is less that the peak value on the spectrum. 

Also check for presence of higher harmonics.

In [ ]:
print(f"{nf} frames of size {Lx, Ly} in the sequence")

In [ ]:
fspec = np.fft.rfft(responsemax)

maxfc = 150 #fourier index above which is irrelevant for the analysis
dcbase = 7   #fourier index below which counts as drift
plt.plot(np.log10(abs(fspec[dcbase:maxfc])))
# plt.plot(np.angle(fspec[dcbase:220]))

mfreq = dcbase + np.argmax(abs(fspec[dcbase:maxfc]))
#shfreq = mfreq + dcbase + np.argmax(abs(fspec[mfreq + dcbase:]))
#thirdfreq = shfreq + dcbase + np.argmax(abs(fspec[shfreq + dcbase:]))

print("Fourier components:", mfreq)
#plt.plot(np.arange(mfreq-1,mfreq+2), abs(fspec[mfreq-1:mfreq+2]), '.')
plt.show()


### Find the average waveform for selected points

TO BE COMPLETED

### Map the fourier components
The first cell is the most computation intensive part. The second cell is only for rendering the outcome.

In [ ]:
## if you're just checking background noise, comment following lines when analyzing oscillating contrasts
# mfreq = 27
# shfreq = 63

# to save testing time, generate harmonic contrast images on smaller areas
region = 1
if region == 1:
    xminf, xmaxf = 500, 700
    yminf, ymaxf = 0, 140
elif region == 2: 
    xminf, xmaxf = 150, 300  #alternative region
    yminf, ymaxf = 0, Ly
elif region == 3: 
    xminf, xmaxf = 300, 450  #alternative region
    yminf, ymaxf = 0, Ly
    
selection = frames[xminf:xmaxf, yminf:ymaxf, :] 
meansel = mean_img[xminf:xmaxf, yminf:ymaxf]
contrast = contrast_img[xminf:xmaxf, yminf:ymaxf]


ffpwidth = 2 #estimated half width of the Fourier peak in the power spectrum
frames_fft = np.fft.rfft(selection, axis=2)/nf
first_harmonic = np.sum(np.power(np.abs(frames_fft[:, :, mfreq-ffpwidth:mfreq+ffpwidth+1]),2), axis=2)

phase = np.angle(frames_fft[:,:, mfreq])/np.pi

In [ ]:
seltag = "corners"+str(xminf)+"x"+str(yminf)+"x"+str(xmaxf)+"x"+str(ymaxf)

fftmax = np.unravel_index(np.argmax(first_harmonic, axis=None), first_harmonic.shape)
# print(fftmax+[xminf, yminf])
fig = plt.figure(figsize=(16,16))

ax1 = plt.subplot(221)
im = ax1.imshow(contrast, extent=(yminf,ymaxf,xminf,xmaxf), origin ='lower')
plt.colorbar(im, ax = ax1)
plt.title("Oscillation contrast")

ax2 = plt.subplot(222)
im = ax2.imshow(np.log10(first_harmonic), vmin=0, extent=(yminf,ymaxf,xminf,xmaxf), origin ='lower')
plt.colorbar(im, ax = ax2)
plt.title("Log of harmonic component")

ax3 = plt.subplot(224)
im = ax3.imshow(phase, cmap='hsv', extent=(yminf,ymaxf,xminf,xmaxf), origin ='lower')
plt.colorbar(im, ax = ax3)
plt.title("Oscillation phase")

ax4 = plt.subplot(223)
im = ax4.imshow(first_harmonic/meansel, cmap='jet', extent=(yminf,ymaxf,xminf,xmaxf), origin ='lower')
plt.colorbar(im, ax = ax4)
plt.title("Harmonic factor divided by intensity ")



# nbins = 40
# hist_harmonic, bin_edges_harmonic = np.histogram(first_harmonic/nf, bins = nbins)

# ax4 = plt.subplot(224)
# ax4.semilogy(bin_edges_harmonic[:nbins], hist_harmonic, '*-')
# ax4.semilogy(bin_edges_var[:nbins], hist_var, '.-')
# plt.title("Distribution of harmonic amplitudes and STD")

plt.show()

# saving some of the data
previewfile = outputdir + mfile.strip(fext)+seltag+'_Summary.png'
fig.savefig(previewfile)

output = outputdir + mfile.strip(fext)+seltag+'_firstharmonic.npy'
np.save(output, first_harmonic)

### Plot intensity vs time at interesting spots

In [ ]:
wx, wy = 5, 10  #estimated half width of speckle size, check that phase is constant over this area
bx, by = meansel.shape[0], meansel.shape[1]
selminmarg = meansel[wx:bx-wx-1, wy:by-wy-1] #used to make sure chosen spekle spots are completely inside the selection
fig = plt.figure(figsize=(18,16))

ax1 = plt.subplot(321)
im = ax1.imshow(np.log10(first_harmonic), vmin=0, extent=(yminf,ymaxf,xminf,xmaxf), origin ='lower')
# im = ax1.imshow(meansel, vmax=2000, extent=(yminf,ymaxf,xminf,xmaxf), origin ='lower')
plt.colorbar(im, ax = ax1)
plt.title("Harmonic power")

mx, my = np.unravel_index(np.argmax(first_harmonic[wx:bx-wx-1, wy:by-wy-1]/selminmarg), selminmarg.shape)
mx = mx + wx
my = my + wy
avg_spot = np.mean(np.mean(selection[mx-wx:mx+wx+1, my-wy:my+wy+1,:]-dark, axis=0), axis=0)
spots = [avg_spot]
#print(mx + xminf,my + yminf)
fh = first_harmonic[mx, my]
si = np.mean(avg_spot)

ax2 = plt.subplot(322)
ax2.plot(avg_spot,'.')
ax1.scatter(my + yminf, mx + xminf, s=8, c='red', marker='o')
plt.title(f"Highest harmonic relative to intensity at {mx + xminf, my + yminf}")
ax2.text(0.05,0.9, "harmonic value: %0.4f, mean: %0.4f"%(fh, si), transform=ax2.transAxes,
      fontsize=16, color = 'red', va='top')

mx, my = np.unravel_index(np.argmax(selminmarg), selminmarg.shape)
# mx, my = 115, 70
mx = mx + wx
my = my + wy
avg_spot = np.mean(np.mean(selection[mx-wx:mx+wx+1, my-wy:my+wy+1,:]-dark, axis=0), axis=0)
#print(mx + xminf,my + yminf)
fh = first_harmonic[mx, my]
si = np.mean(avg_spot)
spots = numpy.concatenate([spots, [avg_spot]], axis=0)

ax3 = plt.subplot(323)
ax3.plot(avg_spot,'.')
ax1.scatter(my + yminf, mx + xminf, s=8, c='orange', marker='o')
plt.title(f"Brightest speckle intensity at {mx + xminf, my + yminf}")
#plt.title(f"handpicked point {mx + xminf, my + yminf}")
ax3.text(0.05,0.9, "harmonic value: %0.4f, mean: %0.4f"%(fh, si), transform=ax3.transAxes,
      fontsize=16, color = 'orange', va='top')

mx, my = np.unravel_index(np.argmax(contrast[wx:bx-wx-1, wy:by-wy-1]), selminmarg.shape)
if region==1: 
    mx, my = 32, 30
elif region ==2:
    mx, my = 70, 18
elif region ==3:
    mx, my = 6, 113
mx = mx + wx
my = my + wy
avg_spot = np.mean(np.mean(selection[mx-wx:mx+wx+1, my-wy:my+wy+1,:]-dark, axis=0), axis=0)
#print(mx + xminf,my + yminf)
fh = first_harmonic[mx, my]
si = np.mean(avg_spot)
spots = numpy.concatenate([spots, [avg_spot]], axis=0)

ax4 = plt.subplot(324)
ax4.plot(avg_spot,'.')
ax1.scatter(my + yminf, mx + xminf, s=8, c='cyan', marker='o')
#plt.title(f"Highest intensity variance at {mx + xminf, my + yminf}")
plt.title(f"handpicked point {mx + xminf, my + yminf}")
ax4.text(0.05,0.9, "harmonic value: %0.4f, mean: %0.4f"%(fh, si), transform=ax4.transAxes,
      fontsize=16, color = 'cyan', va='top')

if region==1: 
    mx, my = 172, 14
elif region ==2:
    mx, my = 100, 122
elif region ==3:
    mx, my = 105, 90
mx = mx + wx
my = my + wy
avg_spot = np.mean(np.mean(selection[mx-wx:mx+wx+1, my-wy:my+wy+1,:]-dark, axis=0), axis=0)
#print(mx + xminf,my + yminf)
fh = first_harmonic[mx, my]
si = np.mean(avg_spot)
spots = numpy.concatenate([spots, [avg_spot]], axis=0)

ax5 = plt.subplot(325)
ax5.plot(avg_spot,'.')
ax1.scatter(my + yminf, mx + xminf, s=8, c='cyan', marker='o')
#plt.title(f"Highest intensity variance at {mx + xminf, my + yminf}")
plt.title(f"handpicked point {mx + xminf, my + yminf}")
ax5.text(0.05,0.9, "harmonic value: %0.4f, mean: %0.4f"%(fh, si), transform=ax5.transAxes,
      fontsize=16, color = 'cyan', va='top')

if region==1: 
    mx, my = 69, 116
elif region ==2:
    mx, my = 62, 140
elif region ==3:
    mx, my = 62, 155
mx = mx + wx
my = my + wy
avg_spot = np.mean(np.mean(selection[mx-wx:mx+wx+1, my-wy:my+wy+1,:]-dark, axis=0), axis=0)
#print(mx + xminf,my + yminf)
fh = first_harmonic[mx, my]
si = np.mean(avg_spot)
spots = numpy.concatenate([spots, [avg_spot]], axis=0)

ax6 = plt.subplot(326)
ax6.plot(avg_spot,'.')
ax1.scatter(my + yminf, mx + xminf, s=8, c='cyan', marker='o')
#plt.title(f"Highest intensity variance at {mx + xminf, my + yminf}")
plt.title(f"handpicked point {mx + xminf, my + yminf}")
ax6.text(0.05,0.9, "harmonic value: %0.4f, mean: %0.4f"%(fh, si), transform=ax6.transAxes,
      fontsize=16, color = 'cyan', va='top')

plt.show()

print(spots.shape)
output = outputdir + mfile.strip(fext) + seltag + '_spots.png'
fig.savefig(output)

output = outputdir + mfile.strip(fext)+ seltag +'_spots.npy'
np.save(output, spots)

### Compare harmonic components on the same spot at different measurement conditions

### Relate the magnitude of the harmonic coefficient to the amplitude of the oscillation

In [ ]:
print(region)